In [1]:
!git clone --recursive https://github.com/borglab/gtsfm.git

Cloning into 'gtsfm'...
remote: Enumerating objects: 37175, done.
remote: Counting objects: 100% (6856/6856), done.
remote: Compressing objects: 100% (1351/1351), done.
remote: Total 37175 (delta 6027), reused 6026 (delta 5500), pack-reused 30319
Receiving objects: 100% (37175/37175), 382.28 MiB | 25.48 MiB/s, done.
Resolving deltas: 100% (26781/26781), done.
Submodule 'thirdparty/LightGlue' (https://github.com/cvg/LightGlue.git) registered for path 'thirdparty/LightGlue'
Cloning into '/kaggle/working/gtsfm/thirdparty/LightGlue'...
remote: Enumerating objects: 382, done.        
remote: Counting objects: 100% (206/206), done.        
remote: Compressing objects: 100% (118/118), done.        
remote: Total 382 (delta 146), reused 93 (delta 88), pack-reused 176        
Receiving objects: 100% (382/382), 17.42 MiB | 21.18 MiB/s, done.
Resolving deltas: 100% (235/235), done.
Submodule path 'thirdparty/LightGlue': checked out 'fe7fb4fa0cffec65e33bf4c2f62a863d5b03433a'


In [2]:
import os
os.chdir('./gtsfm')
!pwd

/kaggle/working/gtsfm


In [3]:
!pip install black
!pip install coverage
!pip install mypy
!pip install pylint
!pip install pytest
!pip install flake8
!pip install isort
  # dask and related
!pip install dask[complete]
!pip install asyncssh
!pip install python-graphviz
  # core functionality and APIs
!pip install matplotlib==3.4.2
!pip install networkx
!pip install numpy
!pip install nodejs
!pip install pandas
!pip install pillow>=8.0.1
!pip install scikit-learn
!pip install seaborn
!pip install scipy
!pip install hydra-core
  # 3rd party algorithms for different modules
!pip install cudatoolkit>=10.2
!pip install pytorch>=1.12.0
!pip install torchvision>=0.13.0
!pip install kornia
  # io
!pip install h5py
!pip install plotly=4.14.3
!pip install tabulate
!pip install simplejson
  # testing
!pip install parameterized
!pip install open3d
!pip install opencv-python>=4.5.4.58
!pip install pydegensac
!pip install colour
!pip install pycolmap==0.4.0
!pip install trimesh[easy]
!pip install gtsam==4.2
!pip install pydot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires ker

In [4]:
!pip install -e .

Obtaining file:///kaggle/working/gtsfm
  Preparing metadata (setup.py) ... done
  Running setup.py develop for gtsfm


In [5]:
!./download_model_weights.sh

Creating ./thirdparty/SuperGluePretrainedNetwork/models/weights
--2024-04-19 12:02:58--  https://github.com/magicleap/SuperGluePretrainedNetwork/raw/master/models/weights/superpoint_v1.pth
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/magicleap/SuperGluePretrainedNetwork/master/models/weights/superpoint_v1.pth [following]
--2024-04-19 12:02:58--  https://raw.githubusercontent.com/magicleap/SuperGluePretrainedNetwork/master/models/weights/superpoint_v1.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5206086 (5.0M) [application/octet-stream]
Saving to: './thirdparty/SuperGluePretrainedNetw

In [6]:
import gtsfm
import gtsam

In [7]:
!mkdir sad
%cd sad
!mkdir images
!cp -r /kaggle/input/random-scenes-for-3d-rec /kaggle/working/sad/images

/kaggle/working/gtsfm/sad
cp: cannot stat '/kaggle/input/random-scenes-for-3d-rec': No such file or directory


In [8]:
from PIL import Image, ExifTags
import os
 
def resize_images_in_directory(input_dir, output_dir, new_width, new_height):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
 
    for filename in os.listdir(input_dir):
        try:
            image = Image.open(os.path.join(input_dir, filename))
 
            resized_image = image.resize((new_width, new_height), Image.LANCZOS)
 
            if "exif" in image.info:
                exif_info = image.info["exif"]
                resized_image.save(os.path.join(output_dir, filename), exif=exif_info)
            else:
                resized_image.save(os.path.join(output_dir, filename))
 
        except Exception as e:
            print(f"Error resizing {filename}: {e}")



In [9]:
import shutil
import os

def move_files(source_dir, dest_dir):
    files = os.listdir(source_dir)
    
    for file in files:
        source_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, file)
        shutil.copy(source_path, dest_path)

# # Example usage:
# source_directory = '/kaggle/input/dataset-home/data'
# destination_directory = '/kaggle/working/images'

# move_files(source_directory, destination_directory)

In [17]:
# %cd /kaggle/working/gtsfm
!rm -r /kaggle/working/images
# mkdir /kaggle/working/images
# !pwd
# %cd .. 
!mkdir /kaggle/working/images
new_width = 1524
new_height = 2032

move_files('/kaggle/input/palace/ordered_v2', '/kaggle/working/images')


In [18]:
import time

start_time = time.time()

!python /kaggle/working/gtsfm/gtsfm/runner/run_scene_optimizer_olssonloader.py >/dev/null --dataset_root /kaggle/working --config_name deep_front_end.yaml --num_workers 4 --output_root /kaggle/working/

end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", (execution_time/60), "Minutes")


/kaggle/working/gtsfm/gtsfm/runner/gtsfm_runner_base.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize_config_module(config_module="gtsfm.configs"):
/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/opt/conda/lib/python3.10/site-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
Execution time: 5.673805518945058 Minutes


In [12]:
# !ls /kaggle/input/random-scenes-for-3d-rec

In [13]:
# from PIL import Image
# from PIL.ExifTags import TAGS
# import os 
# def get_exif_data(image_path):
#     counter = 0
#     for i in os.listdir(image_path):
#         exif_data = {}
#         try:
#             print(image_path + "/" + i)
#             with Image.open(image_path + "/" + i) as img:
#                 # Check if the image contains EXIF metadata
#                 if hasattr(img, '_getexif'):
#                     # Get the EXIF metadata
#                     exif_info = img._getexif()
#                     if exif_info:
#                         for tag, value in exif_info.items():
#                             tag_name = TAGS.get(tag, tag)
#                             exif_data[tag_name] = value
#         except:
#             print("An error occurred while processing the image.")
#         print(counter)
#         counter+=1

# # Example usage:?
# image_path = '/kaggle/input/dataset-home/data'  # Replace with the path to your JPEG image
# exif_data = get_exif_data(image_path)
# for tag, value in exif_data.items():
#     print(f"{tag}: {value}")


In [14]:
# !cat /kaggle/working/gtsfm/gtsfm/loader/loader_base.py